In [1]:
import os
import json
import numpy as np
import cv2
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
Image.MAX_IMAGE_PIXELS = None
from skimage import io

In [2]:
import datetime

from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:

# references/detection
!git clone https://github.com/pytorch/vision.git
!cd vision
!git checkout v0.8.2

!cp ./vision/references/detection/utils.py ./
!cp ./vision/references/detection/transforms.py ./
!cp ./vision/references/detection/coco_eval.py ./
!cp ./vision/references/detection/engine.py ./
!cp ./vision/references/detection/coco_utils.py ./

!pip install cython
# Install pycocotools, the version by default in Colab
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -U albumentations
!pip install -U opencv-python

#Copy and unify the train and validation datasets into one folder for images and another for labels
!mkdir ./train
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/train/images/. ./train/images/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/valid/images/. ./train/images/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/train/labels/. ./train/labels/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/valid/labels/. ./train/labels/

fatal: destination path 'vision' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


ERROR: Invalid requirement: "'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'": Expected package name at the start of dependency specifier
    'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
    ^
Hint: = is not a valid operator. Did you mean == ?


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import utils
# import transforms as T  # Use OpenCV or Pillow for image transformations instead
import cv2
import time
# from albumentations.pytorch.transforms import ToTensorV2  # Use PyTorch or other framework-specific modules
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import stats  # Alternative for statistical functions
from PIL import Image  # Alternative for basic image transformations
import random


In [5]:
class CraterDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(self.root, "images"))))
        self.annots = list(sorted(os.listdir(os.path.join(self.root, "labels"))))
        self.classes = ['Background','Crater']
        
    # Converts boundry box formats, this version assumes single class only!
    def convert_box_cord(self,bboxs, format_from, format_to, img_shape):
        if format_from == 'normxywh':
            if format_to == 'xyminmax':
                xw = bboxs[:, (1, 3)] * img_shape[1]
                yh = bboxs[:, (2, 4)] * img_shape[0]
                xmin = xw[:, 0] - xw[:, 1] / 2
                xmax = xw[:, 0] + xw[:, 1] / 2
                ymin = yh[:, 0] - yh[:, 1] / 2
                ymax = yh[:, 0] + yh[:, 1] / 2
                coords_converted = np.column_stack((xmin, ymin, xmax, ymax))

        return coords_converted

    def __getitem__(self, idx):
        # load images and boxes
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        annot_path = os.path.join(self.root, "labels", self.annots[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img= img/255.0

        # retrieve bbox list and format to required type,
        # if annotation file is empty, fill dummy box with label 0
        if os.path.getsize(annot_path) != 0:
            bboxs = np.loadtxt(annot_path, ndmin=2)
            bboxs = self.convert_box_cord(bboxs, 'normxywh', 'xyminmax', img.shape)
            num_objs = len(bboxs)
            bboxs = torch.as_tensor(bboxs, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        else:
            bboxs = torch.as_tensor([[0, 0, 640, 640]], dtype=torch.float32)
            labels = torch.zeros((1,), dtype=torch.int64)
            iscrowd = torch.zeros((1,), dtype=torch.int64)

        area = (bboxs[:, 3] - bboxs[:, 1]) * (bboxs[:, 2] - bboxs[:, 0])
        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = bboxs
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            sample = self.transforms(image=img,
                                     bboxes=target['boxes'],
                                     labels=labels)
        img = sample['image']
        target['boxes'] = torch.tensor(sample['bboxes'])
        target['labels'] = torch.tensor(sample['labels'])
        if target['boxes'].ndim == 1:
            target['boxes'] = torch.as_tensor([[0, 0, 640, 640]], dtype=torch.float32)
            target['labels'] = torch.zeros((1,), dtype=torch.int64)
        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
def get_model_bbox(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [7]:
def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc',min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

In [8]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [24]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import random

class CraterBoulderDataset:
    def __init__(self, images_dir, labels_dir):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_paths = sorted(glob.glob(os.path.join(images_dir, '*.jpg')))  # Adjust file extension as per your images

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = os.path.join(self.labels_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')

        # Load image
        img = plt.imread(img_path)

        # Load labels
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_label = int(parts[0])  # Assuming class label is present
                        x_center = float(parts[1]) * img.shape[1]  # Convert to absolute coordinates
                        y_center = float(parts[2]) * img.shape[0]
                        width = float(parts[3]) * img.shape[1]
                        height = float(parts[4]) * img.shape[0]
                        labels.append({
                            'class_label': class_label,
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })

        return img, labels

# Function to plot image with bounding boxes and save to file
def plot_img_bbox_save(img, labels, save_path):
    fig, a = plt.subplots(1, 1)
    fig.set_size_inches(8, 8)
    a.imshow(img)

    for label in labels:
        x_center = label['x_center']
        y_center = label['y_center']
        width = label['width']
        height = label['height']

        x_min = x_center - width / 2
        y_min = y_center - height / 2
        x_max = x_center + width / 2
        y_max = y_center + height / 2

        rect = patches.Rectangle((x_min, y_min), (x_max - x_min), (y_max - y_min),
                                 edgecolor='r', facecolor='none', clip_on=False)  # Changed color to red
        a.add_patch(rect)
        a.annotate('Crater', (x_min, y_min - 10), color='red', weight='bold',  # Changed color to red
                   fontsize=8, ha='left', va='top')

    plt.axis('off')  # Hide axes
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Function to save labels and crater count to CSV
# Function to save labels to CSV with the number of craters detected in each image
def save_labels_to_csv(dataset, csv_path):
    rows = []
    for idx in range(len(dataset)):
        img_path = dataset.image_paths[idx]
        labels = dataset[idx][1]  # Get labels for the image
        num_craters = len(labels)  # Number of craters in the image

        for label in labels:
            rows.append({
                'image_path': os.path.basename(img_path),
                'class_label': label['class_label'],
                'x_center': label['x_center'],
                'y_center': label['y_center'],
                'width': label['width'],
                'height': label['height'],
                'num_craters': num_craters
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(csv_path, index=False)

# Example usage
images_dir = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\images'
labels_dir = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\labels'
output_dir = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\output_images'
csv_file = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\labels.csv'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

dataset = CraterBoulderDataset(images_dir, labels_dir)

# Plot and save all images with annotations
for idx in range(len(dataset)):
    img, labels = dataset[idx]
    file_name = os.path.basename(dataset.image_paths[idx])
    save_path = os.path.join(output_dir, file_name)
    plot_img_bbox_save(img, labels, save_path)

# Save labels to CSV
save_labels_to_csv(dataset, csv_file)

print(f"All images with annotations have been saved to {output_dir}.")
print(f"All labels have been saved to {csv_file}.")


All images with annotations have been saved to D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\output_images.
All labels have been saved to D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\labels.csv.


In [25]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

class CraterBoulderDataset:
    def __init__(self, images_dir, labels_dir):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_paths = sorted(glob.glob(os.path.join(images_dir, '*.jpg')))  # Adjust file extension as per your images

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = os.path.join(self.labels_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')

        # Load image
        img = plt.imread(img_path)

        # Load labels
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_label = int(parts[0])  # Assuming class label is present
                        x_center = float(parts[1]) * img.shape[1]  # Convert to absolute coordinates
                        y_center = float(parts[2]) * img.shape[0]
                        width = float(parts[3]) * img.shape[1]
                        height = float(parts[4]) * img.shape[0]
                        labels.append({
                            'class_label': class_label,
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })

        return img, labels

# Example usage
images_dir = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\images'
labels_dir = r'D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\labels'

dataset = CraterBoulderDataset(images_dir, labels_dir)

# Plotting function with 'Crater' annotation and red color
def plot_img_bbox(img, labels):
    fig, a = plt.subplots(1, 1)
    fig.set_size_inches(8, 8)
    a.imshow(img)

    for label in labels:
        x_center = label['x_center']
        y_center = label['y_center']
        width = label['width']
        height = label['height']

        x_min = x_center - width / 2
        y_min = y_center - height / 2
        x_max = x_center + width / 2
        y_max = y_center + height / 2

        rect = patches.Rectangle((x_min, y_min), (x_max - x_min), (y_max - y_min),
                                 edgecolor='r', facecolor='none', clip_on=False)  # Changed color to red
        a.add_patch(rect)
        a.annotate('Crater', (x_min, y_min - 10), color='red', weight='bold',  # Changed color to red
                   fontsize=8, ha='left', va='top')

    plt.show()

# Print 3 random examples to check annotations with red color
for idx in random.sample(range(len(dataset)), 3):
    img, labels = dataset[idx]
    plot_img_bbox(img, labels)


C:\Users\Prayag Chawla\AppData\Local\Temp\ipykernel_30844\875038580.py:75: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [26]:
import torch
from torch.utils.data import DataLoader, Dataset

class CraterBoulderDataset(Dataset):
    def __init__(self, images_dir, labels_dir):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_paths = sorted(glob.glob(os.path.join(images_dir, '*.jpg')))  # Adjust file extension as needed

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = os.path.join(self.labels_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')

        # Load image
        img = plt.imread(img_path)
        img = torch.tensor(img).permute(2, 0, 1).float()  # Convert to PyTorch tensor and reorder dimensions

        # Load labels
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_label = int(parts[0])  # Assuming class label is present
                        x_center = float(parts[1]) * img.shape[2]  # Convert to absolute coordinates
                        y_center = float(parts[2]) * img.shape[1]
                        width = float(parts[3]) * img.shape[2]
                        height = float(parts[4]) * img.shape[1]
                        labels.append({
                            'class_label': class_label,
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })

        # Convert to PyTorch format
        target = {
            'boxes': torch.tensor([
                [label['x_center'] - label['width'] / 2, 
                 label['y_center'] - label['height'] / 2, 
                 label['x_center'] + label['width'] / 2, 
                 label['y_center'] + label['height'] / 2]
                for label in labels
            ], dtype=torch.float32),
            'labels': torch.tensor([label['class_label'] for label in labels], dtype=torch.int64)
        }

        return img, target

# Create the dataset and DataLoader
dataset = CraterBoulderDataset(images_dir, labels_dir)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: x)  # Adjust batch_size as needed


In [27]:
import os
import glob

def convert_labels(images_dir, labels_dir):
    # Get all image paths
    image_paths = glob.glob(os.path.join(images_dir, '*.jpg'))  # Adjust file extension if needed
    
    for img_path in image_paths:
        # Corresponding label file
        label_path = os.path.join(labels_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as file:
                lines = file.readlines()
                
            # Convert each line
            with open(label_path, 'w') as file:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        class_id, x_center, y_center, width, height = parts
                        file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
                    else:
                        print(f"Invalid format in {label_path}: {line}")

# Define paths
images_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/images'
labels_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/labels'

# Convert label files
convert_labels(images_dir, labels_dir)


In [28]:
import yaml
from ultralytics import YOLO

# Define paths
train_images_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/images'
train_labels_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/labels'
val_images_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/images'  # Reuse train images
val_labels_dir = 'D:/Prayag Files/TIET/Extras/Internship/Ongoing/hexagon/craters/train/labels'  # Reuse train labels
data_cfg_path = 'data.yaml'

# Create YOLOv5 dataset YAML configuration
data_cfg = {
    'train': train_images_dir,
    'val': val_images_dir,
    'nc': 2,  # Number of classes
    'names': ['crater', 'boulder']  # Class names
}

# Save data configuration to a YAML file
with open(data_cfg_path, 'w') as f:
    yaml.dump(data_cfg, f)

# Load YOLOv5 model
model = YOLO('yolov5s.pt')  # Load a pre-trained YOLOv5 model

# Train the model
model.train(
    data=data_cfg_path,
    epochs=10,  # Number of epochs
    imgsz=640,  # Image size
    batch=4,    # Batch size
    project='runs_train',  # Directory for saving results
    name='exp',  # Experiment name
    exist_ok=True  # Overwrite existing runs
)

# Save the trained model
model_path = 'runs_train/exp/weights/best.pt'
model.save(model_path)

print(f"Model saved to {model_path}")


PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



New https://pypi.org/project/ultralytics/8.2.61 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.60  Python-3.11.0 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov5s.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_train, name=exp, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\labels.cache... 98 images, 9 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning D:\Prayag Files\TIET\Extras\Internship\Ongoing\hexagon\craters\train\labels.cache... 98 images, 9 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<?, ?it/s]

...


10 epochs completed in 0.623 hours.
Optimizer stripped from runs_train\exp\weights\last.pt, 18.5MB
Optimizer stripped from runs_train\exp\weights\best.pt, 18.5MB

Validating runs_train\exp\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.11.0 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
YOLOv5s summary (fused): 193 layers, 9,112,310 parameters, 0 gradients, 23.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:56<00:00,  4.34s/it]


                   all         98        681        0.7      0.573      0.665      0.391
                crater         89        681        0.7      0.573      0.665      0.391
Speed: 3.1ms preprocess, 537.3ms inference, 0.0ms loss, 11.8ms postprocess per image
Results saved to runs_train\exp


lr/pg0,▃▅▇█▇▆▅▃▂▁
lr/pg1,▃▅▇█▇▆▅▃▂▁
lr/pg2,▃▅▇█▇▆▅▃▂▁
metrics/mAP50(B),▅▁▄▅▆▅▇▇██
metrics/mAP50-95(B),▅▁▃▄▅▅▆▇██
metrics/precision(B),▆▁▄▅▆▅▆▇██
metrics/recall(B),▆▁▇▆▆▆▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▃▄▃▃▃▂▁▁


Model saved to runs_train/exp/weights/best.pt
